In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from IPython.display import Javascript, display, clear_output, HTML

display(HTML("<style>.container { width:100% !important;}</style>"))

import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [2]:
%run data_loading.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from typing import Literal

In [4]:
fixtures_df = get_fixtures_df()
teams_df = get_teams_df()

In [5]:
def merge_fixtures_and_teams(
    fixtures_df: pd.DataFrame, teams_df: pd.DataFrame
) -> pd.DataFrame:
    for home_away in ["home", "away"]:
        renaming_dict = {
            "team_id": f"{home_away}_team_id",
            "team_short_name": f"{home_away}_team_short_name",
            "team_name": f"{home_away}_team_name",
        }
        home_away_teams_df = teams_df.rename(columns=renaming_dict)[
            list(renaming_dict.values())
        ].copy()
        fixtures_df = fixtures_df.merge(
            home_away_teams_df, how="left", on=[f"{home_away}_team_id"]
        )
    return fixtures_df

In [6]:
fixtures_df = merge_fixtures_and_teams(fixtures_df=fixtures_df, teams_df=teams_df)

In [7]:
fixtures_df.head().T

,0,1,2,3,4
global_match_id,2561895,2561896,2561897,2561900,2561899
match_id,1,2,3,6,5
game_week,1,1,1,1,1
is_finished,False,False,False,False,False
is_finished_provisional,False,False,False,False,False
match_start_time,2025-08-15T19:00:00Z,2025-08-16T11:30:00Z,2025-08-16T14:00:00Z,2025-08-16T14:00:00Z,2025-08-16T14:00:00Z
is_provisional_start_time,False,False,False,False,False
is_started,False,False,False,False,False
home_team_id,12,2,6,18,17
away_team_id,4,15,10,3,19


In [15]:
import numpy as np

In [16]:
def get_team_fixtures_df(fixtures_df: pd.DataFrame) -> pd.DataFrame:
    team_fixtures_dfs = []
    for home_away, away_home in [("home", "away"), ("away", "home")]:
        team_fixtures_df = (
            fixtures_df[
                [
                    "match_id",
                    "game_week",
                    "is_finished",
                    "is_started",
                    "match_start_time",
                    f"{home_away}_team_id",
                    f"{home_away}_team_name",
                    f"{home_away}_team_short_name",
                    f"{home_away}_team_score",
                    f"{away_home}_team_score",
                ]
            ]
            .rename(
                columns={
                    f"{home_away}_team_id": "team_id",
                    f"{home_away}_team_name": "team_name",
                    f"{home_away}_team_short_name": "team_short_name",
                    f"{home_away}_team_score": "goals_for",
                    f"{away_home}_team_score": "goals_against",
                }
            )
            .copy()
        )
        team_fixtures_df["home_away"] = home_away
        condlist = [
            team_fixtures_df["goals_for"] > team_fixtures_df["goals_against"],
            team_fixtures_df["goals_for"] == team_fixtures_df["goals_against"],
            team_fixtures_df["goals_for"] < team_fixtures_df["goals_against"],
        ]
        team_fixtures_df["result"] = np.select(
            condlist=condlist,
            choicelist=["W", "D", "L"],
            default=None,
        )
        team_fixtures_df["points"] = np.select(
            condlist=condlist,
            choicelist=[3, 1, 0],
            default=np.nan,
        )
        team_fixtures_dfs.append(team_fixtures_df)
    return pd.concat(team_fixtures_dfs).sort_values(
        by=["match_id", "home_away"], ascending=[True, False], ignore_index=True
    )

In [17]:
team_fixtures_df = get_team_fixtures_df(fixtures_df=fixtures_df)

In [18]:
team_fixtures_df

,match_id,game_week,is_finished,is_started,match_start_time,team_id,team_name,team_short_name,goals_for,goals_against,home_away,result,points
0,1,1,False,False,2025-08-15T19:00:00Z,12,Liverpool,LIV,None,None,home,None,NaN
1,1,1,False,False,2025-08-15T19:00:00Z,4,Bournemouth,BOU,None,None,away,None,NaN
2,2,1,False,False,2025-08-16T11:30:00Z,2,Aston Villa,AVL,None,None,home,None,NaN
3,2,1,False,False,2025-08-16T11:30:00Z,15,Newcastle,NEW,None,None,away,None,NaN
4,3,1,False,False,2025-08-16T14:00:00Z,6,Brighton,BHA,None,None,home,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,378,38,False,False,2026-05-24T15:00:00Z,7,Chelsea,CHE,None,None,away,None,NaN
756,379,38,False,False,2026-05-24T15:00:00Z,18,Spurs,TOT,None,None,home,None,NaN
757,379,38,False,False,2026-05-24T15:00:00Z,9,Everton,EVE,None,None,away,None,NaN
758,380,38,False,False,2026-05-24T15:00:00Z,19,West Ham,WHU,None,None,home,None,NaN


In [28]:
def get_league_table_df(
    team_fixtures_df: pd.DataFrame,
    started_or_finished: Literal["started", "finished"] = "started",
) -> pd.DataFrame:
    team_fixtures_df = team_fixtures_df.sort_values(
        by=["match_id", "home_away"], ascending=[True, False], ignore_index=True
    )
    league_table_df = (
        team_fixtures_df
        .groupby(by=["team_name"], as_index=False)
        .agg(
            matches_played=(f"is_{started_or_finished}", "sum"),
            won=("result", lambda x: sum(x == "W")),
            drawn=("result", lambda x: sum(x == "D")),
            lost=("result", lambda x: sum(x == "L")),
            points=("points", "sum"),
            goals_for=("goals_for", "sum"),
            goals_against=("goals_against", "sum"),
            form=(
                "result",
                lambda x: x[team_fixtures_df[f"is_{started_or_finished}"]]
                .tail(5)
                .tolist(),
            ),
        )
    )
    league_table_df["goal_difference"] = (
        league_table_df["goals_for"] - league_table_df["goals_against"]
    )
    league_table_df = league_table_df.sort_values(
        by=["points", "goal_difference", "goals_for"],
        ascending=False,
        ignore_index=True,
    )
    league_table_df["position"] = range(1, 21)
    return league_table_df

In [29]:
league_table_df = get_league_table_df(team_fixtures_df=team_fixtures_df)

In [30]:
league_table_df

,team_name,matches_played,won,drawn,lost,points,goals_for,goals_against,form,goal_difference,position
0,Arsenal,0,0,0,0,0.0,0,0,[],0,1
1,Aston Villa,0,0,0,0,0.0,0,0,[],0,2
2,Bournemouth,0,0,0,0,0.0,0,0,[],0,3
3,Brentford,0,0,0,0,0.0,0,0,[],0,4
4,Brighton,0,0,0,0,0.0,0,0,[],0,5
5,Burnley,0,0,0,0,0.0,0,0,[],0,6
6,Chelsea,0,0,0,0,0.0,0,0,[],0,7
7,Crystal Palace,0,0,0,0,0.0,0,0,[],0,8
8,Everton,0,0,0,0,0.0,0,0,[],0,9
9,Fulham,0,0,0,0,0.0,0,0,[],0,10
